<a href="https://colab.research.google.com/github/deconasser/UdemyCourse/blob/main/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow
!pip install segmentation_models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.7 MB/s eta 0:00:00


In [3]:
!pip install albumentations

In [1]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import albumentations as A
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from albumentations import Compose, HorizontalFlip, RandomRotate90

import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from albumentations import (
    Compose,
    RandomRotate90,
    Flip,
    Transpose,
    ElasticTransform,
    GridDistortion,
    OpticalDistortion,
    RandomBrightnessContrast,
    HorizontalFlip,
    VerticalFlip,
    RandomGamma,
    RGBShift,
)

Segmentation Models: using `tf.keras` framework.


In [2]:
def preprocess_gt(gt_image):
    gt_image = np.array(gt_image, dtype=np.uint8)
    num_classes = 3
    gt_one_hot = np.zeros((gt_image.shape[0], gt_image.shape[1], num_classes), dtype=np.float32)

    # Lớp 0: Màu đen (background)
    gt_one_hot[:, :, 0] = (gt_image == 0).all(axis=-1).astype(np.float32)
    # Lớp 1: Màu xanh lá cây (lành tính)
    gt_one_hot[:, :, 1] = (gt_image[:, :, 1] == 255).astype(np.float32) # channel 1 (G) == 255
    # Lớp 2: Màu đỏ (ác tính)
    gt_one_hot[:, :, 2] = (gt_image[:, :, 0] == 255).astype(np.float32) # channel 0 (R) == 255

    return gt_one_hot

def load_data(image_path, mask_path, img_size=(256, 256)):
    images = []
    masks = []

    image_files = sorted(os.listdir(image_path))
    mask_files = sorted(os.listdir(mask_path))

    for img_file, mask_file in zip(image_files, mask_files):
        img = load_img(os.path.join(image_path, img_file), target_size=img_size)
        img = img_to_array(img) / 255.0

        mask = load_img(os.path.join(mask_path, mask_file), target_size=img_size)
        mask = img_to_array(mask)

        mask = preprocess_gt(mask)

        images.append(img)
        masks.append(mask)

    images = np.array(images, dtype=np.float32)
    masks = np.array(masks, dtype=np.float32)

    return images, masks

# Đường dẫn tới dữ liệu
train_images_path = '/content/drive/MyDrive/Unet/data/train'
train_masks_path = '/content/drive/MyDrive/Unet/data/train_gt'

In [3]:
train_images, train_masks = load_data(train_images_path, train_masks_path)

In [4]:
test = load_img("/content/drive/MyDrive/Unet/data/train_gt/0081835cf877e004e8bfb905b78a9139.jpeg")
test = img_to_array(test)
test = preprocess_gt(test)
print(test[0][0])

[1. 0. 0.]


In [5]:
def augment_data(images, masks):
  aug = Compose([
      HorizontalFlip(p=0.5),
      RandomRotate90(p=0.5),
      A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.5),
      A.RandomBrightnessContrast(p=0.5)
  ])
  augmented_images = []
  augmented_masks = []
  for img, mask in zip(images, masks):
      augmented = aug(image=img, mask=mask)
      augmented_images.append(augmented['image'])
      augmented_masks.append(augmented['mask'])
  return np.array(augmented_images), np.array(augmented_masks)

In [6]:
augmented_images, augmented_masks = augment_data(train_images, train_masks)

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

def weighted_dice_loss(y_true, y_pred):
  smooth = 1.0
  class_weights = tf.constant([0.5, 2.0, 2.0], dtype=tf.float32)

  # Apply class weights
  y_true_weighted = y_true * class_weights
  y_pred_weighted = y_pred * class_weights

  y_true_f = tf.keras.backend.flatten(y_true_weighted)
  y_pred_f = tf.keras.backend.flatten(y_pred_weighted)
  intersection = tf.keras.backend.sum(y_true_f * y_pred_f)

  return 1 - (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


def iou_metric(y_true, y_pred):
  y_true_f = tf.keras.backend.flatten(y_true)
  y_pred_f = tf.keras.backend.flatten(y_pred)
  intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
  union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
  return intersection / union

def dice_coefficient(y_true, y_pred):
  smooth = 1.0
  y_true_f = tf.keras.backend.flatten(y_true)
  y_pred_f = tf.keras.backend.flatten(y_pred)
  intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
  return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def unet_model(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)


    # Decoder

    up7 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv4))
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Final layer with softmax activation
    segmentation_output = layers.Conv2D(3, 1, activation='softmax', name='segmentation')(conv9)

    model = models.Model(inputs=inputs, outputs=segmentation_output)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss=weighted_dice_loss, metrics=[iou_metric, dice_coefficient])

    return model

model = unet_model()


In [8]:
import datetime
# Tạo thư mục lưu trữ kết quả huấn luyện
log_dir = os.path.join("/content/drive/MyDrive/Unet/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(log_dir, exist_ok=True)

# Callbacks
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
    verbose=1
)

tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1
)

callbacks = [checkpoint_callback, early_stopping_callback, tensorboard_callback]


In [9]:
X_train, X_val, y_train, y_val = train_test_split(augmented_images, augmented_masks, test_size=0.2, random_state=42)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
50/50 [==============================] - ETA: 0s - loss: 0.6190 - iou_metric: 0.7557 - dice_coefficient: 0.8211
Epoch 1: val_loss improved from inf to 0.52417, saving model to best_model.keras
50/50 [==============================] - 93s 1s/step - loss: 0.6190 - iou_metric: 0.7557 - dice_coefficient: 0.8211 - val_loss: 0.5242 - val_iou_metric: 0.9270 - val_dice_coefficient: 0.9620
Epoch 2/50
50/50 [==============================] - ETA: 0s - loss: 0.5213 - iou_metric: 0.9358 - dice_coefficient: 0.9668
Epoch 2: val_loss did not improve from 0.52417
50/50 [==============================] - 44s 874ms/step - loss: 0.5213 - iou_metric: 0.9358 - dice_coefficient: 0.9668 - val_loss: 0.5242 - val_iou_metric: 0.9270 - val_dice_coefficient: 0.9620
Epoch 3/50
50/50 [==============================] - ETA: 0s - loss: 0.5213 - iou_metric: 0.9358 - dice_coefficient: 0.9668
Epoch 3: val_loss did not improve from 0.52417
50/50 [==============================] - 46s 915ms/step - loss: 0.5213 